# 3. Generation-based MRC

## 강의소개

- 3강에서는 생성기반 기계독해에 대해 배워보겠습니다.
- 생성기반으로 기계독해를 푼다는 것의 의미를 이해하고, 어떻게 생성기반 기계독해를 풀 수 있을지 알아보겠습니다.
- 2강에서와 마찬가지로 모델 학습에 필요한 전처리 단계, 생성기반 모델 학습 단계, 그리고 최종적으로 답을 얻어내는 세 단계로 나눠 생성기반 기계독해를 푸는 방법에 대해 배워볼 예정입니다.

<br>

## Further Reading

- [Introducing BART](https://sshleifer.github.io/blog_v2/jupyter/2020/03/12/bart.html)
- [BART: Denoising Sequence-to-Sequence Pre-training for Natural Language Generation, Translation, and Comprehension](https://arxiv.org/abs/1910.13461)
- [Exploring the Limits of Transfer Learning with a Unified Text-to-Text Transformer (T5)](https://arxiv.org/abs/1910.10683)

<br>

## 3.1 Generation-based MRC

- Generation-based MRC 개념 및 개요
- Generation-based MRC 평가 방법

<br>

### 3.1.1 MRC 문제를 푸는 방법

#### 3.1.1.1 Extraction-based MRC

- 지문(context) 내 답의 위치를 예측
- 분류 문제 (classification)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1lE3y_INeWcPEqHEHkUkp_i2GPYz72YhI' width=800/>

- 모든 Extraction-based MRC 문제는 Generation-based MRC 문제로 치환할 수 있다.

<br>

#### 3.1.1.2  Generation-based MRC

- 주어진 지문과 질의(question)을 보고 답변을 생성
- 생성 문제 (generation)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1lEPaJgz7JVedBYkGtdVRo56-EZgbtBpK' width=800/>

- Generation-based MRC 문제는 Extraction-based MRC 문제로 치환하기 힘들다.
  - 정답이 문장내에 존재하지 않은 경우 치환할 수 없음

<br>

### 3.1.2 Generation-based MRC 평가 방법

- 동일한 extractive answer datasets $\Rightarrow$ Extraction-based MRC와 동일한 평가 방법을 사용 (recap)
- 하지만 좀 더 생성 문제와 일반적으로 접근하려면 ROUGE난 BLEU Score를  평가 지표로 사용해야 한다.

<br>

#### 3.1.2.1 Exact Match (EM) Score

- EM = 1 when (Characters of the prediction) == (Characters of ground-truth) Otherwise, EM = 0

<br>

#### 3.1.2.2 F1 Score

- 예측한 답과 ground-truth 사이의 token overlap을 F1으로 계산

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1lHZS0AfX2WfqHyKNGzgUmH_LHu0PZKDf' width=800/>

<br>

### 3.1.3 Generation-based MRC Overview

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1lLCDTNZX0POAWNQHHvy8C9Q3831yT33k' width=800/>

- input은 Extraction-based MRC와 유사하다.
- 모델이 출력으로 정답 텍스트를 생성한다.
- 일종의 seq2seq 모델이라고 볼 수 있다.
  - 모든 PLM이 seq2seq을 할수 있는 것은 아니다.
    - BERT의 경우 인코딩 단계만 있고 디코더가 없기 때문에 Generation-based MRC 문제에 적용할 수 없다.

<br>

### 3.1.4 Generation-based MRC & Extraction-based MRC 비교

#### 3.1.4.1 MRC 모델 구조

- Generation-based MRC
  - Seq-to-seq PLM(Pre-trained Language Model) 구조
- Extraction-based MRC
  - PLM + Classifier 구조

<br>

#### 3.1.4.2 Loss 계산을 위한 답의 형태 / Prediction의 형태

- Generation-based MRC
  - 텍스트를 디코딩할 때 Teacher Forcing 방식으로 학습을 진행
  - Free-form text 형태
- Extraction-based MRC
  - 위치에 대한 확률분포를 Negative Log Likelihood를 계산하는 방식
  - 지문 내 답의 위치
  - F1 계산을 위해 text로의별도 변환 과정이 필요

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1lLtKAT-gLHEph51NLJ-unfK-twl1u8pN' width=800/>

<br>

## 3.2 Pre-processing

- 입력 표현
- 출력 표현

<br>

- Generation-based MRC의 processing은 Extraction-based MRC보다 더 간단하다.
  - input 단계는 완전히 동일
  - post-processing 만 차이가 있음

<br>

### 3.2.1 입력 표현 - 데이터 예시

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1lMQ0dcr4cW6BwFJ_M9Bw58wlzf22ocky' width=800/>

<br>

### 3.2.2 입력 표현 - 토큰화

- Tokenization: 텍스트를 의미를 가진 작은 단위로 나눈 것 (형태소)
- Extraction-based MRC와 같이 **WordPiece Tokenizer**를 사용함
  - WordPiece Tokenizer 사전 학습 단계에서 먼저 학습에 사용한 전체 데이터 집합(코퍼스)에 대해서 구축되어 있어야 함
  - 구축 과정에서 미리 각 단어 토큰들에 대해 순서대로 번호(인덱스)를 부여해 둠
- Tokenizer는 입력 텍스트를 토큰화한 뒤, 각 토큰을 미리 만들어둔 단어 사전에 따라 인덱스로 변환함

- WordPiece Tokenizer 사용 예시
  - 질문
    - `"미국 군대 내 두번째로 높은 직위는 무엇인가?"`
  - 토큰화된 질문
    - `['미국', '군대', '내', '두번째', '##로', '높은', '직', '##위는', '무엇인가', '?']`
    - `##`은 해당 토큰이 이전 토큰과 붙어 있다는 것을 의미 (디코딩 시 사용)
  - 인덱스로 바뀐 질문
    - `[101, 23545, 8910, 14423, 8996, 9102, 48506, 11261, 55600, 9707, 19855, 11018, 9294, 119137, 12030, 11287, 136, 102]`

- 인덱스로 바뀐 질문을 보통 `input_ids`(또는 `input_token_ids`)로 부른다.
- 모델의 기본 입력은 `input_ids`만 필요하나, 그 외 추가적인 정보가 필요하다.

<br>

### 3.2.3 입력 표현 - Special Token

- 학습 시에만 사용되며 단어 자체의 의미는 가지지 않는 특별한 토큰
- ex) SOS(Start Of Sentence), EOS(End Of Sentence), CLS, SEP, PAD, UNK, ... 등

- Extraction-based MRC
  - CLS, SEP, PAD 토큰 사용
- Generation-based MRC
  - PAD 토큰 사용
  - CLS, SEP 토큰 또한 사용할 수 있으나, 대신 자연어를 이용하여 정해진 텍스트 포맷(format)으로 데이터를 생성
  - 경우에 따라서는 CLS 대신 `question`, `context` 등의 스페셜 토큰을 사용할 수도 있다. (사용하는 모듈에 따라 다름)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1lNlAac5qGLjJoStgyeEckvMJP4DcKebr' width=800/>

<br>

### 3.2.4  입력 표현 - additional information

#### 3.2.4.1 Attention mask

- Extraction-based MRC와 똑같이 어텐션 연산을 수행할 지 결정하는 어텐션 마스크 존재

<br>

#### 3.2.4.2 Token Type ids

- 질문(question)과 지문(context)를 구분
- BERT와 달리 BART에서는 입력 시퀀스에 대한 구분이 없어 `token_type_ids`가 존재하지 않음
- 따라서 Extraction-based MRC와 달리 입력에 `token_type_ids`가 들어가지 않음
  - SEP 토큰을 이용해 질문과 지문을 구분할 수 있으므로 `token_type_ids`가 제공되지 않아도 어느정도 모델이 구분할 수 있다.
  - 초창기에는 `token_type_ids`를 통해 직접적으로 구분해주려고 했지만 나중엔 중요하지 않다고 판단되어 사용하지 않는 모델들이 존재한다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1lQ5YxH0fb7KMEC8pon2nrUyLiwTeU5oF' width=800/>

<br>

### 3.2.5 출력 표현 - 정답 출력

- Sequence of token ids

<br>

- Extraction-based MRC에선 텍스트를 생성해내는 대신 시작/끝 토큰의 위치를 출력하는 것이 모델의 최종 목표였음
- Generation-based MRC는 그보다 조금 더 어려운 실제 텍스트를 생성하는 과제를 수행
- 전체 시퀀스의 각 위치마다 모델이 아는 모든 단어들 중 하나의 단어를 맞추는 classification 문제

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1lQ70Ym3itRdZaTaaGAolo8NRHBMJAJJX' width=800/>

<br>

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1lSujs_ZWcmDcDGDRgayRWGhBEtY2PF-a' width=800/>

- 일반적인 디코딩 방법론이라 볼 수 있다.

<br>

## 3.3 Model

- BART Encoder & Decoder
- Pre-training BART

<br>

### 3.3.1 BART

- 기계 독해, 기계 번역, 요약, 대화 등 sequence to sequence 문제의 pre-training을 위한 denosing autoencoder

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1lTg_pA-yEpNj0J9js-hjtU-tdti8duOm' width=800/>

- BERT는 인코더만 존재
- GPT는 디코더만 존재
- BART는 인코더, 디코더 모두 존재

<br>

- BART의 pre-training 방식
  - noise를 injection하고 noise가 없는 원래 버전을 reconstruction
  - 그렇기 때문에 **denosing autoencoder**라는 표현을 사용하기도 한다.

<br>

### 3.3.2 BART Encoder & Decoder

- BART의 인코더는 BERT처럼 bi-directional
- BART의 디코더는 GPT처럼 uni-directional (autoregressive)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1lVa29M2DwzluM7l0qnQbH5tScbfsKDfg' width=600/>

<br>

### 3.3.3 Pre-training BART

- BART는 텍스트에 노이즈를 주고 원래 텍스트를 복구하는 문제를 푸는 것으로 pre-training 한다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1lXBzcOXzEEuOK5T2JnGOVqPZmgq69xWi' width=600/>

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1lZi9CFmBoRiXShYoUGZ0zOh3ybKg1Xu7' width=500/>

- 입력 토큰 중 일부를 없앰 (corrupted)
- 원래 토큰을 복원하는 쪽으로 pre-training 진행

<br>

## 3.4 Post-processing

- Decoding
- Searching

<br>

### 3.4.1 Decoding

- 디코더에서 이전 스텝에서 나온 출력이 다음 스텝의 입력으로 들어간다. (autoregressive)
- 맨 처음 입력은 문장 시작을 뜻하는 스페셜 토큰

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1lcMLlcbKDHM4kX8SqMNIOD7tS3HR6Kp-' width=300/>

<br>

### 3.4.2 Searching

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1lcNNDLDRLD2NEKfJEQ6-kRncdQz_BqLJ' width=800/>

- `<s>` : 문장의 시작을 의미하는 스페셜 토큰

<br>

- 디코딩 방법론에는 여러 가지가 존재한다.
  - Greedy Search, Exhaustive Search, Beam Search

<br>

- Greedy Search
  - 시작하는 `<s>`에서 가장 likely한 토큰들을 선택해나가는 방식
  - 효율적이고 빠르게 문장을 생성하는 방법
  - 생성 자체는 빠르지만 각 단계의 토큰이 이후에 안 좋은 선택이 될 수 있어서 안좋은 output을 낼 수 있다. (early decision making)

<br>

- Exhaustive Search
  - 모든 가능성을 전부 고려한다.
  - 하지만 가능한 가지의 숫자가 time step에 비례해서 exponential하게 늘어남
  - 그렇기 때문에 문장의 길이가 조금만 길어져도 불가능한 방법론이다.
  - 또한 vocabulary size가 조금만 커져도 마찬가지로 불가능한 방법론이다.

<br>

- Beam Search
  - 일반적으로 모든 모델들이 채택하는 방법론
  - Exhaustive Search를 하긴 하되 각 time step마다 점수가 높은 top-k만 유지하는 방식
  - 이렇게 하면 보유하고 있는 candidate의 리스트는 일정한 숫자로 유지된다.